# Training of classifier with TMVA

## import necessary libraries 

In [1]:
import ROOT

Welcome to JupyROOT 6.15/02


In [2]:
from ROOT import gSystem, gROOT, gApplication, TFile, TTree, TCut, TCanvas, TGraph

In [3]:
from ROOT import TMVA
%jsroot on

### Define training variables

In [4]:
InputVar = [
    "max_eta"              ,
    "lep_Pt_1"             ,
    "Mll01"                , 
    "minDeltaR_LJ_0"       ,
    "minDeltaR_LJ_1"       ,
    "MET_RefFinal_et"      ,
    "nJets_OR_T"           ,
    "nJets_OR_T_MV2c10_70" ,
    "lep_flavour"          
]

### Create instance of TMVA factory
see TMVA/macros/TMVAClassification.C for more factory options

 some examples of options for the factory
"!V:ROC:!Correlations:!Silent:Color:!DrawProgressBar:AnalysisType=Classification" ); 



In [5]:
TMVA.Tools.Instance()

<ROOT.TMVA::Tools object at 0x7f90a2d74180>

In [6]:
outputFile = TFile.Open("TMVAOutputCV.root", "RECREATE");

In [7]:
factory = TMVA.Factory( "TMVAClassification", outputFile, 
                             "!V:ROC:!Correlations:!Silent:Color:!DrawProgressBar:AnalysisType=Classification")        

In [8]:
dataloader = TMVA.DataLoader("dataset")
# as well name for output folder with relevant plots

Define the input variables that shall be used for the classifier training

In [9]:
InputVarSize = len(InputVar);
#print(InputVarSize)
for ivar in range(len(InputVar)):
    #print(InputVar[i])
    dataloader.AddVariable(InputVar[ivar],'F');

Load signal and background files to training factory

In [10]:
signal = ROOT.TChain("nominal")
signal.Add("../../Files/skimmed/ttH.root")

1

In [ ]:
for branch in signal.GetListOfBranches():
    print(branch)

In [11]:
background = ROOT.TChain("nominal")
background.Add("../../Files/skimmed/ttW.root")
#background.GetListOfBranches()

1

In [12]:
signalWeight     = 1.0
backgroundWeight = 1.0

register trees

In [13]:
dataloader.AddSignalTree    ( signal,     signalWeight     )
dataloader.AddBackgroundTree( background, backgroundWeight )

DataSetInfo              : [dataset] : Added class "Signal"
                         : Add Tree nominal of type Signal with 69902 events
DataSetInfo              : [dataset] : Added class "Background"
                         : Add Tree nominal of type Background with 86754 events


In [14]:
#Set individual event weights (the variables must exist in the original TTree)
dataloader.SetSignalWeightExpression( "weightS" );
dataloader.SetBackgroundWeightExpression( "weightS" );

In [15]:
# Apply additional cuts on the signal and background sample. 
mycutSig = TCut( "" ) 
mycutBkg = TCut( "" ) 
# <- keep empty as samples were specifically prepared during slimming: 
# https://github.com/grevtsovkirill/tthml_perf/blob/master/skim/GN2_light/ugly_2lss_sel.cpp#L3810


In [16]:
dataloader.PrepareTrainingAndTestTree( mycutSig, mycutBkg,
                                        "nTrain_Signal=0:nTrain_Background=0:SplitMode=Random:NormMode=NumEvents:V" )

In [24]:
#factory.BookMethod( dataloader, TMVA.Types.kBDT, "BDT",
#                   "!H:!V:NTrees=1000:MinNodeSize=2.5%:BoostType=Grad:Shrinkage=0.10:UseBaggedBoost:BaggedSampleFraction=0.5:nCuts=20:MaxDepth=2" )

setdefBDTprops = "!H:!V:NTrees=1000:MinNodeSize=2.5%:BoostType=Grad:Shrinkage=0.10:UseBaggedBoost:BaggedSampleFraction=0.5:nCuts=20:MaxDepth=2"


 https://root.cern.ch/root/htmldoc/guides/tmva/TMVAUsersGuide.pdf page 22  methods definitions

In [17]:
# Boosted Decision Trees, Gradient Boost from tutorials (https://github.com/root-project/root/blob/master/tutorials/tmva/TMVAClassification.C#L504)
factory.BookMethod( dataloader, TMVA.Types.kBDT, "BDT",setdefBDTprops)
                   #"!H:!V:NTrees=1000:MinNodeSize=2.5%:BoostType=Grad:Shrinkage=0.10:UseBaggedBoost:BaggedSampleFraction=0.5:nCuts=20:MaxDepth=2" )

#agrees to the tth default:
# Method_Opt = "!H:!V:NTrees=1000:MinNodeSize=1.5%:BoostType=Grad:Shrinkage=0.10:
# UseBaggedBoost:BaggedSampleFraction=0.5:nCuts=20:MaxDepth=2";

<ROOT.TMVA::MethodBDT object ("BDT") at 0x7f90a1f9e600>

Factory                  : Booking method: BDT
                         : 
                         : the option NegWeightTreatment=InverseBoostNegWeights does not exist for BoostType=Grad
                         : --> change to new default NegWeightTreatment=Pray
                         : Parsing option string: 
                         : ... "nTrain_Signal=0:nTrain_Background=0:SplitMode=Random:NormMode=NumEvents:V"
                         : The following options are set:
                         : - By User:
                         :     SplitMode: "Random" [Method of picking training and testing events (default: random)]
                         :     NormMode: "NumEvents" [Overall renormalisation of  event-by-event weights used in the training (NumEvents: average weight of 1 per event, independently for signal and background; EqualNumEvents: average weight of 1 per event for signal, and sum of weights for background equal to sum of weights for signal)]
                        

In [ ]:
#factory.BookMethod( dataloader, TMVA.Types.kSVM, "SVM", "Gamma=1:Tol=0.1:VarTransform=Norm" )
# even gama1, tol 0.1 still too slow
#"Gamma=0.25:Tol=0.001:VarTransform=Norm - do not converge locally - too granular

In [ ]:
# low performance 0.620 wrt 0.736 for BDT
#factory.BookMethod( dataloader, TMVA.Types.kRuleFit, "RuleFit",
#                           "H:V:RuleFitModule=RFTMVA:Model=ModRuleLinear:MinImp=0.001:RuleMinDist=0.001:NTrees=20:fEventsMin=0.01:fEventsMax=0.5:GDTau=-1.0:GDTauPrec=0.01:GDStep=0.01:GDNSteps=10000:GDErrScale=1.02" )


In [18]:
   # Train MVAs
factory.TrainAllMethods()

Factory                  : Train all methods
Factory                  : [dataset] : Create Transformation "I" with events from all classes.
                         : 
                         : Transformation, Variable selection : 
                         : Input : variable 'max_eta' <---> Output : variable 'max_eta'
                         : Input : variable 'lep_Pt_1' <---> Output : variable 'lep_Pt_1'
                         : Input : variable 'Mll01' <---> Output : variable 'Mll01'
                         : Input : variable 'minDeltaR_LJ_0' <---> Output : variable 'minDeltaR_LJ_0'
                         : Input : variable 'minDeltaR_LJ_1' <---> Output : variable 'minDeltaR_LJ_1'
                         : Input : variable 'MET_RefFinal_et' <---> Output : variable 'MET_RefFinal_et'
                         : Input : variable 'nJets_OR_T' <---> Output : variable 'nJets_OR_T'
                         : Input : variable 'nJets_OR_T_MV2c10_70' <---> Output : variable 'nJets_OR_T_

In [19]:
    # Test MVAs
factory.TestAllMethods()
    

Factory                  : Test all methods
Factory                  : Test method: BDT for Classification performance
                         : 
BDT                      : [dataset] : Evaluation of BDT on testing sample (78328 events)
                         : Elapsed time for evaluation of 78328 events: 2.47 sec       


In [20]:
    # Evaluate MVAs
factory.EvaluateAllMethods()    
    


Factory                  : Evaluate all methods
Factory                  : Evaluate classifier: BDT
                         : 
BDT                      : [dataset] : Loop over test events and fill histograms with classifier response...
                         : 
TFHandler_BDT            :             Variable                    Mean                    RMS            [        Min                    Max ]
                         : -----------------------------------------------------------------------------------------------------------------------
                         :              max_eta:                1.2453               0.56708   [             0.0056135                2.4999 ]
                         :             lep_Pt_1:                50414.                29917.   [                20000.            4.5473e+05 ]
                         :                Mll01:            1.4737e+05            1.1223e+05   [                12004.            1.5992e+06 ]
               

In [21]:
    # Save the output.
outputFile.Close()

In [22]:
c2=factory.GetROCCurve(dataloader)
c2.Draw()

In [23]:
factory.GetROCIntegral(dataloader,"BDT")
# applying weights changes AUROC 0.736 to 0.726

0.7265699853415943

In [ ]:
#factory.GetROCIntegral(dataloader,"RuleFit")

In [ ]:
#factory.GetROCCurve (dataloader,"RuleFit") #

### Variable importance 

In [25]:
vi = TMVA.VariableImportance(dataloader)

#vi.BookMethod(TMVA.Types.kBDT, "BDT",
#"!V:NTrees=5:MinNodeSize=2.5%:MaxDepth=2:BoostType=AdaBoost:AdaBoostBeta=0.5:UseBaggedBoost:BaggedSampleFraction=0.5:SeparationType=GiniIndex:nCuts=20" )
vi.BookMethod(TMVA.Types.kBDT, "BDT",setdefBDTprops)

vi.SetType(TMVA.kShort)

vi.Evaluate()

                         : Evaluation done.


In [ ]:
results=vi.GetResults()
results.Print()
results.Draw()